In [1]:
! pip install azure-ai-ml -U 
! pip install mltable azureml-dataprep[pandas] -U 
! pip install azureml-fsspec -U

zsh:1: no matches found: azureml-dataprep[pandas]


In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [3]:
subscription_id = "Your Azure Subscription ID" 
resource_group = "Your Azure Machine Learning Service Workspace Resource Group" 
workspace = "Your Azure Machine Learning Service Workspace Name"

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
from azure.ai.ml.entities import OneLakeDatastore, OneLakeArtifact

In [6]:
artifact = OneLakeArtifact(
    name=<Lakehouse ID>, 
    type="lake_house"
)
store = OneLakeDatastore(
    name="onelake_lh_for_azureml",
    description="Credential-less OneLake datastore.",
    endpoint="msit-onelake.dfs.fabric.microsoft.com",
    artifact=artifact,
    one_lake_workspace_name=<One Lake workspace name>,
)

Class OneLakeArtifact: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class OneLakeDatastore: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [7]:
ml_client.create_or_update(store)

OneLakeDatastore({'type': <DatastoreType.ONE_LAKE: 'OneLake'>, 'name': 'onelake_lh_for_azureml', 'description': 'Credential-less OneLake datastore.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/901e11c6-42ed-4fcc-875a-6bd77390af5b/resourceGroups/AIGroup/providers/Microsoft.MachineLearningServices/workspaces/KinfeyAOAIWS/datastores/onelake_lh_for_azureml', 'Resource__source_path': None, 'base_path': '/home/lokinfey/Dev/Code/AOAI/FabricWithAzureML', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7feab7517a00>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x7feab75178b0>, 'artifact': {'name': 'db6d5797-6c48-4318-abe2-eaaa14ff4136', 'type': 'lake_house'}, 'one_lake_workspace_name': 'c162308d-0437-46b9-9db0-1a28ae4fc460', 'endpoint': 'msit-onelake.dfs.fabric.microsoft.com'})

In [10]:
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azureml.fsspec import AzureMachineLearningFileSystem

In [11]:
uri = 'azureml://subscriptions/<Your Azure Subscription ID >/resourcegroups/<Your Azure Machine Learning Service Resource Group>/workspaces/<Your Azure Machine Learning Service Workspace Name>/datastores/onelake_lh_for_azureml'

# create the filesystem
fs = AzureMachineLearningFileSystem(uri)

In [12]:
fs.ls()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Resolving access token for scope "https://storage.azure.com/.default" using identity of type "USER".


['Files', 'TableMaintenance', 'Tables']

In [14]:
with fs.open('Files/csv/sales.csv') as f:
    data = f.readlines()
    print(data[0:5])
    f.close()

[b'SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount\r\n', b'SO43701,1,2019-07-01,Christy Zhu,christy12@adventure-works.com,"Mountain-100 Silver, 44",1,3399.99,271.9992\r\n', b'SO43704,1,2019-07-01,Julio Ruiz,julio1@adventure-works.com,"Mountain-100 Black, 48",1,3374.99,269.9992\r\n', b'SO43705,1,2019-07-01,Curtis Lu,curtis9@adventure-works.com,"Mountain-100 Silver, 38",1,3399.99,271.9992\r\n', b'SO43700,1,2019-07-01,Ruben Prasad,ruben10@adventure-works.com,"Road-650 Black, 62",1,699.0982,55.9279\r\n']


In [15]:

from azure.ai.ml.entities import Data

In [16]:
csv_path = 'azureml://datastores/onelake_lh_for_azureml/paths/Files/csv'
my_csv_data = Data(
        path=csv_path,
        type=AssetTypes.URI_FOLDER,
        description="demo",
        name="csv_data_source",
        version="1.0.0"
)

ml_client.data.create_or_update(my_csv_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'csv_data_source', 'description': 'demo', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/901e11c6-42ed-4fcc-875a-6bd77390af5b/resourceGroups/AIGroup/providers/Microsoft.MachineLearningServices/workspaces/KinfeyAOAIWS/data/csv_data_source/versions/1.0.0', 'Resource__source_path': None, 'base_path': '/home/lokinfey/Dev/Code/AOAI/FabricWithAzureML', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7feac99a0e80>, 'serialize': <msrest.serialization.Serializer object at 0x7feab69e8fd0>, 'version': '1.0.0', 'latest_version': None, 'path': 'azureml://subscriptions/901e11c6-42ed-4fcc-875a-6bd77390af5b/resourcegroups/AIGroup/workspaces/KinfeyAOAIWS/datastores/onelake_lh_for_azureml/paths/Files/csv/', 'datastore': None})

In [17]:
import pandas as pd

In [18]:
csv_data = ml_client.data.get("csv_data_source", version="1.0.0")


In [19]:
import mltable

In [20]:

path = {
  'folder': csv_data.path
}

tbl = mltable.from_delimited_files(paths=[path])

In [22]:
df = pd.read_csv( csv_data.path + '/sales.csv')

In [23]:
df

,SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount
0,SO43701,1,2019-07-01,Christy Zhu,christy12@adventure-works.com,"Mountain-100 Silver, 44",1,3399.9900,271.9992
1,SO43704,1,2019-07-01,Julio Ruiz,julio1@adventure-works.com,"Mountain-100 Black, 48",1,3374.9900,269.9992
2,SO43705,1,2019-07-01,Curtis Lu,curtis9@adventure-works.com,"Mountain-100 Silver, 38",1,3399.9900,271.9992
3,SO43700,1,2019-07-01,Ruben Prasad,ruben10@adventure-works.com,"Road-650 Black, 62",1,699.0982,55.9279
4,SO43703,1,2019-07-01,Albert Alvarez,albert7@adventure-works.com,"Road-150 Red, 62",1,3578.2700,286.2616
...,...,...,...,...,...,...,...,...,...
32713,SO63093,2,2021-12-31,Rebekah Martin,rebekah20@adventure-works.com,Patch Kit/8 Patches,1,2.2900,0.1832
32714,SO63113,1,2021-12-31,Carolyn Alonso,carolyn29@adventure-works.com,"Road-550-W Yellow, 48",1,1120.4900,89.6392
32715,SO63113,2,2021-12-31,Carolyn Alonso,carolyn29@adventure-works.com,"Short-Sleeve Classic Jersey, S",1,53.9900,4.3192
32716,SO63066,1,2021-12-31,Teresa Navarro,teresa10@adventure-works.com,Mountain Tire Tube,1,4.9900,0.3992
